In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import dateutil.parser as dp
from datetime import datetime
from datetime import timedelta  
import time
from datetime import timezone
session = Session()

#converts pyton datatime to unix milisecond timestamp utc timezone
def to_utcunix(x):
    return int(x.replace(tzinfo=timezone.utc).timestamp()) * 1000

def get_data(start, batch_size, end = datetime.now(), symbol = "BTCUSDT", interval = "1m"):
    batch_start = start
    
    while True:
        batch_end = batch_start + timedelta(minutes = batch_size)#+ timedelta(seconds = granularity * (batch_size ))
        
        if(batch_end > end):
            batch_end = end - timedelta(minutes = 1)
        
            if(batch_start >= end- timedelta(minutes = 1)):
                break
        
        base = "https://api.binance.com/api/v3"

        print(batch_start, batch_end)
        
        symbol = "BTCUSDT"
        interval = "1m"
        
        url = base + "/klines?symbol={}&interval={}&startTime={}&endTime={}&limit={}".format(symbol, interval,
                                                                                     to_utcunix(batch_start)
                                                                                    , to_utcunix(batch_end), batch_size)
        
        data = session.get(url).json()
        batch_start = batch_end
        
        #sleep to stay under binance maximum call rate
        time.sleep(0.101)
        
        yield pd.DataFrame.from_dict(data)

def gen_csv(start, filename, end = datetime.now(), symbol = "BTCUSDT", interval = "1m"):
    batch_size = 1000
    d = get_data(start, batch_size, end = end, symbol = symbol, interval = interval)
    for i in d:
        i.to_csv(filename + ".csv", mode = 'a', header = False)

def gen_csv_year(year, symbol, interval = "1m"):
    gen_csv(datetime(year,1,1) , "{}_{}".format(year, symbol) , end = datetime(year + 1,1,1))
    

#for i in range(2016, 2021):
#    gen_csv_year(i, "ETHUSDT")    


In [2]:
gen_csv_year(2018, "BTCUSDT")

2018-01-01 00:00:00 2018-01-01 16:40:00
2018-01-01 16:40:00 2018-01-02 09:20:00
2018-01-02 09:20:00 2018-01-03 02:00:00
2018-01-03 02:00:00 2018-01-03 18:40:00
2018-01-03 18:40:00 2018-01-04 11:20:00
2018-01-04 11:20:00 2018-01-05 04:00:00
2018-01-05 04:00:00 2018-01-05 20:40:00
2018-01-05 20:40:00 2018-01-06 13:20:00
2018-01-06 13:20:00 2018-01-07 06:00:00
2018-01-07 06:00:00 2018-01-07 22:40:00
2018-01-07 22:40:00 2018-01-08 15:20:00
2018-01-08 15:20:00 2018-01-09 08:00:00
2018-01-09 08:00:00 2018-01-10 00:40:00
2018-01-10 00:40:00 2018-01-10 17:20:00
2018-01-10 17:20:00 2018-01-11 10:00:00
2018-01-11 10:00:00 2018-01-12 02:40:00
2018-01-12 02:40:00 2018-01-12 19:20:00
2018-01-12 19:20:00 2018-01-13 12:00:00


KeyboardInterrupt: 